In [12]:
# Import all models to use them within bagging technique
# Known that bagging work with only one base estimator cloning it within bootsreapped data with replacement
from Libraries.LinearRegression import *
from Libraries.LogisticRegression import *
from Libraries.KNN import *
from Libraries.SVM import *
from Libraries.NaiveBayes import *
from Libraries.DecisionTree import *
from Libraries.RandomForest import *
from Libraries.Voting import *
from Libraries.AdaBoost import *
from Libraries.GradientBoost import *
import sys, importlib, shutil
shutil.rmtree("Libraries/__pycache__", ignore_errors=True)
sys.modules.pop("Libraries.RandomForest", None)
importlib.invalidate_caches()

In [13]:
import numpy as np
# We need this copy for cloning base estimator
import copy
# For parallelism and working with multiple of cores 
from joblib import Parallel, delayed

# Bagging class for Classification and Regression
# We work with the base class then inheritence happen to build the classification or regression bagging
class BaggingBase():

    # Initialization
    def __init__(self, base_estimator = None, n_estimators = 10, random_state = None, n_jobs = None):
        # Here we gonna define the base estimator later in classification or regression
        # Others is the same default paramerts of API
        # n_jobs=None or 1 is the same and -1 means using all cores
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.models = []


    # We have helper functions
    # Bootstrapping samples function
    def _bootstrap_sample(self, X, y):
        # Here we fetch the number of samples from x 
        # X is combination of samples x features
        n_samples = X.shape[0]

        # Every time we select random numbers of indice with enabling replacement option so we can use same indice in more models
        # So we can get the sample 0, 1 or multiple of times 
        # Here we choose randomly from all samples and we need the size of bootstrapped data = size of orginal data that is why replace option should be applied
        indices = np.random.choice(n_samples, n_samples, replace=True)

        # Here we return the bootstrapped data
        return X[indices], y[indices]

    # Clone function
    def _clone(self):
        # Here we use the library of copy to return clone of base_estimator
        # The idea here to siolate object per model
        # It is only for learning phase
        return copy.deepcopy(self.base_estimator)
    
    # Modeling function
    # We will have one fit function per every model then combine all fits in general fit function for all memebers
    # Fit single model
    def _fit_single_model(self, X, y):
        # First we recieve the bootstrapped data to model it from given X and y
        X_samples, y_samples = self._bootstrap_sample(X, y)
        
        # Get cloned copy of base_estimator
        # Every time we gonna pass the object of base estimator
        cloned_model = self._clone()

        # We use the fit function for specific single model which was already built from scratch before
        cloned_model.fit(X_samples, y_samples)

        # Every time we return the fitted model
        return cloned_model
    
    # Fit all models function
    # Here we gather all fitted models 
    def fit(self, X, y):
        # We set random state wuth the number user entered and seed it
        if self.random_state is not None:
            np.random.seed(self.random_state)

        # Loop over estimator number collecting all fitted models in one place to be ready for test phase
        # We call _fit_single_model every time
        self.models = Parallel(n_jobs=self.n_jobs)(
            delayed(self._fit_single_model)(X, y) for estimator in range(self.n_estimators)
        )    
        
        # We return nothing
        return self
    
    # Predict probabilities
    # This function is ready to use in soft oting later
    def predict_proba(self, X):
        probas = Parallel(n_jobs=self.n_jobs)(
            delayed(model.predict_proba)(X) for model in self.models
        )
        probas = np.array(probas)
        return np.mean(probas, axis=0)

    # The same structure of fit will be built for predict
    # We will have one predict function per every model then combine all predicts in general predict function for all memebers
    # Predict single model
    def _predict_single_model(self, cloned_model, X):
        # We use the predict function for specific single model which was already built from scratch before
        # We use test data
        return cloned_model.predict(X)

    # Precit all models function
    # Here we gather all precited models 
    def predict(self, X):
        # Loop over model collecting all predictions in one place to be ready for aggregation later using cpu cores (parallelism)
        # We call _predict_single_model every time
        # to apply predict we need to concatenate with fit first this is gathered in self.models
        # Here every fitted model work with all samples 
        # So each sample have multiple predictions for all models
        predictions = Parallel(n_jobs=self.n_jobs)(
            delayed(self._predict_single_model)(model, X) for model in self.models
        )    
        
        # Preditction as numpy array
        predictions = np.array(predictions)
        
        # After collecting predictions we are ready for aggregation
        return self._aggregate(predictions)

    # Aggregation function    
    # Aggregation to decide the final output
    # Here we create just abstract so after inheritence each of classification and regression has its own methodoly of aggregation for final prediction
    def _aggregate(self, predictions):
        raise NotImplementedError  

# Bagging ensemble Classifier class
class BaggingClassifier(BaggingBase):

    # Intialization
    # We pass known numbers of paramters to parent class and also at the same time get known numbers of paramters
    def __init__(self, base_estimator=None, n_estimators=10, random_state=None, n_jobs=1):
        
        # Default estimator for classification as API
        if base_estimator is None:
            base_estimator = DecisionTreeClassifier()

        super().__init__(base_estimator, n_estimators, random_state, n_jobs)

    # Aggregation function    
    def _aggregate(self, predictions):

        # Get number of samples as every sample wich will be columns as every column represents sample and indexes represent models  
        n_samples = predictions.shape[1]

        # Gather all predictions in one array and make sure its data type the same as the source predictions array
        # Intialize
        final_predictions = np.zeros(n_samples, dtype=predictions.dtype)

        # Apply majority voting 
        # Loop over all samples to get the most voted label
        for sample in range(n_samples):
            # Featch models of samples values
            # Every sample represents column
            votes = predictions[:, sample]

            # Here we get unique classes then counting them
            # We return two related arrays of classes and labels 
            # One for unique values and other for unique counts
            # So return the index of laregest count will be equivelant to the label of largest count
            classes, counts = np.unique(votes, return_counts=True)
            majority_voting = classes[np.argmax(counts)]
            final_predictions[sample] = majority_voting

        # Return the final predictions of the test data
        return final_predictions
    
    # Score
    def score(self, X, y):
        # Return accuracy score 
        return np.mean(self.predict(X) == y)

# Bagging ensemble Tree Regressor class
class BaggingRegressor(BaggingBase):

    # Intialization
    # We pass known numbers of paramters to parent class and also at the same time get known numbers of paramters
    def __init__(self, base_estimator=None, n_estimators=10, random_state=None, n_jobs=1):
        
        # Default estimator for classification as API
        if base_estimator is None:
            base_estimator = DecisionTreeRegressor()

        super().__init__(base_estimator, n_estimators, random_state, n_jobs)

    # Aggregation function 
    def _aggregate(self, predictions):
        # We just return the mean of every sample wich will be columns as every column represents sample and indexes represent trees
        return np.mean(predictions, axis=0)  
  
    # Score
    def score(self, X, y):
        return np.mean((self.predict(X) - y) ** 2)

In [14]:
# Sample data
# Create number of rows and random x and y matrices
m = 100
# Use rand then multiple by 2 to make sure the samples values are between 0 and 2 this would make sure we simulate feature scaling
# Here we create 2 features
X1 = 2 * np.random.rand(m, 1)
X2 = 2 * np.random.rand(m, 1)
# Combine features to create X matrix
X = np.column_stack((X1, X2))

# Y value will split to intercept + value + noise from 1st feature [row, columns] to simulate real data doing regression
y_output = 4 + 3 * X[:, 0] + np.random.randn(m)

# Y value will be Continuous target (intercept + value + noise) from 1st feature [row, columns] then convert to binary classes doing classification
y_continuous = 4 + 3 * X[:, 0] + np.random.randn(m)
threshold = np.mean(y_continuous)
y_labels = (y_continuous > threshold).astype(int)

# Test data with 2 rows
X_new = np.array([[1,2], [3,4], [4,3]])

# Apply default bagging with all possible tasks 
# Tasks
tasks = ['Classification', 'Regression']
for task in tasks:
    if task == 'Classification':
        # Get Default Bagging Classifier object
        bg_model = BaggingClassifier(n_jobs=-1)
        bg_model.fit(X, y_labels)
        y_pred = bg_model.predict(X_new)
        score_train = bg_model.score(X, y_labels) * 100
        print(f'Task: {task}')
        print(f'Model: Defualt Bagging Classifier')
        print(f'Predictions for {X_new.tolist()}: {y_pred}')
        print(f'Accuracy score on training data: {score_train:.2f} %')

    elif task == 'Regression':
        # Get Default Bagging Regressor object
        bg_model = BaggingRegressor(n_jobs=-1)
        bg_model.fit(X, y_output)
        y_pred = bg_model.predict(X_new)
        score_train = bg_model.score(X, y_output) * 100
        print(f'Task: {task}')
        print(f'Model: Defualt Bagging Regressor')
        print(f'Predictions for {X_new.tolist()}: {np.round(y_pred,2)}')
        print(f'R^2 score on training data: {score_train:.2f} %')
    print('-'*40)    

# Apply differnet estimators bagging with all possible tasks 
# Tasks
classification_estimators = [LogisticRegression(method='Gradient Descent'), LogisticRegression(method='Ridge'), LogisticRegression(method='Lasso'), 
                             KNN(k=3, task='Classification'), 
                             SVC(kernel='Linear'), SVC(kernel='Polynomial', gamma=0.5), 
                             SVC(kernel='RBF', gamma=0.5), SVC(kernel='Sigmoid'),
                             SVC(kernel='Linear',decision_function_shape='ovo'), SVC(kernel='Polynomial', gamma=0.5,decision_function_shape='ovo'), 
                             SVC(kernel='RBF', gamma=0.5, decision_function_shape='ovo'), SVC(kernel='Sigmoid',decision_function_shape='ovo'),
                             SVC(kernel='Linear',decision_function_shape='ovr'), SVC(kernel='Polynomial', gamma=0.5,decision_function_shape='ovr'), 
                             SVC(kernel='RBF', gamma=0.5, decision_function_shape='ovr'), SVC(kernel='Sigmoid',decision_function_shape='ovr'),
                             GaussianNB(), MultinomialNB(alpha=1.0), BernoulliNB(alpha=1.0),
                             DecisionTreeClassifier(max_depth=3),
                             RandomForestClassifier(max_depth=3, n_jobs=-1),
                             VotingClassifier(estimators= [LogisticRegression(method='Gradient Descent'), LogisticRegression(method='Ridge'), LogisticRegression(method='Lasso'), 
                                                           KNN(k=3, task='Classification'), 
                                                           SVC(kernel='Linear'), SVC(kernel='Polynomial', gamma=0.5), 
                                                           SVC(kernel='RBF', gamma=0.5), SVC(kernel='Sigmoid'),
                                                           SVC(kernel='Linear',decision_function_shape='ovo'), SVC(kernel='Polynomial', gamma=0.5,decision_function_shape='ovo'), 
                                                           SVC(kernel='RBF', gamma=0.5, decision_function_shape='ovo'), SVC(kernel='Sigmoid',decision_function_shape='ovo'),
                                                           SVC(kernel='Linear',decision_function_shape='ovr'), SVC(kernel='Polynomial', gamma=0.5,decision_function_shape='ovr'), 
                                                           SVC(kernel='RBF', gamma=0.5, decision_function_shape='ovr'), SVC(kernel='Sigmoid',decision_function_shape='ovr'),
                                                           GaussianNB(), MultinomialNB(alpha=1.0), BernoulliNB(alpha=1.0),
                                                           DecisionTreeClassifier(max_depth=3),
                                                           RandomForestClassifier(max_depth=3, n_jobs=-1),
                                                           AdaBoostClassifier(),
                                                           GradientBoostClassifier()
                                                           ], n_jobs=1),
                             VotingClassifier(estimators= [LogisticRegression(method='Gradient Descent'), LogisticRegression(method='Ridge'), LogisticRegression(method='Lasso'), 
                                                           KNN(k=3, task='Classification'), 
                                                           SVC(kernel='Linear'), SVC(kernel='Polynomial', gamma=0.5), 
                                                           SVC(kernel='RBF', gamma=0.5), SVC(kernel='Sigmoid'),
                                                           SVC(kernel='Linear',decision_function_shape='ovo'), SVC(kernel='Polynomial', gamma=0.5,decision_function_shape='ovo'), 
                                                           SVC(kernel='RBF', gamma=0.5, decision_function_shape='ovo'), SVC(kernel='Sigmoid',decision_function_shape='ovo'),
                                                           SVC(kernel='Linear',decision_function_shape='ovr'), SVC(kernel='Polynomial', gamma=0.5,decision_function_shape='ovr'), 
                                                           SVC(kernel='RBF', gamma=0.5, decision_function_shape='ovr'), SVC(kernel='Sigmoid',decision_function_shape='ovr'),
                                                           GaussianNB(), MultinomialNB(alpha=1.0), BernoulliNB(alpha=1.0),
                                                           DecisionTreeClassifier(max_depth=3),
                                                           RandomForestClassifier(max_depth=3, n_jobs=-1),
                                                           AdaBoostClassifier(),
                                                           GradientBoostClassifier()
                                                           ], n_jobs=1, voting='soft'),
                             AdaBoostClassifier(),
                             GradientBoostClassifier()                                                        
                             ]
regression_estimators = [LinearRegression(method='OLS'), LinearRegression(method='Normal'), LinearRegression(method='Gradient Descent'), 
                         LinearRegression(method='Ridge'), LinearRegression(method='Ridge-Gradient Descent'), LinearRegression(method='Lasso-Gradient Descent'),
                         KNN(k=3, task='Regression'), 
                         SVR(kernel='Linear'), SVR(kernel='Polynomial', gamma=0.5), 
                         SVR(kernel='RBF', gamma=0.5), SVR(kernel='Sigmoid'),
                         DecisionTreeRegressor(max_depth=3),
                         RandomForestRegressor(max_depth=3, n_jobs=-1),
                         VotingRegressor(estimators= [LinearRegression(method='OLS'), LinearRegression(method='Normal'), LinearRegression(method='Gradient Descent'), 
                                                      LinearRegression(method='Ridge'), LinearRegression(method='Ridge-Gradient Descent'), LinearRegression(method='Lasso-Gradient Descent'),
                                                      KNN(k=3, task='Regression'), 
                                                      SVR(kernel='Linear'), SVR(kernel='Polynomial', gamma=0.5), 
                                                      SVR(kernel='RBF', gamma=0.5), SVR(kernel='Sigmoid'),
                                                      DecisionTreeRegressor(max_depth=3),
                                                      RandomForestRegressor(max_depth=3, n_jobs=-1),
                                                      AdaBoostRegressor(),
                                                      GradientBoostRegressor()
                                                      ], n_jobs=1),
                         AdaBoostRegressor(),
                         GradientBoostRegressor()                             
                         ]

tasks = ['Classification', 'Regression']
for task in tasks:
    if task == 'Classification':
        for estimator in classification_estimators:
            # Get Model of Bagging Classifier object
            bg_model = BaggingClassifier(base_estimator=estimator, n_jobs=1)
            bg_model.fit(X, y_labels)
            y_pred = bg_model.predict(X_new)
            score_train = bg_model.score(X, y_labels) * 100
            print(f'Task: {task}')
            # Get model class name
            model_name = estimator.__class__.__name__
            # Get model attributes
            attributes = estimator.__dict__
            # Convert dictionary to list then to string
            formatted_params = ", ".join([f"{k}='{v}'" if isinstance(v, str) else f"{k}={v}" for k, v in attributes.items()])
            # Concatenate model class name with its attributes
            estimator_name = f"{model_name}({formatted_params})"
            print(f"Model: {estimator_name}")
            print(f'Predictions for {X_new.tolist()}: {y_pred}')
            print(f'Accuracy score on training data: {score_train:.2f} %')
            print('-'*40)  

    elif task == 'Regression':
        for estimator in regression_estimators:
            # Get Model of Bagging Regressor object
            bg_model = BaggingRegressor(base_estimator=estimator, n_jobs=1)
            bg_model.fit(X, y_output)
            y_pred = bg_model.predict(X_new)
            score_train = bg_model.score(X, y_output) * 100
            print(f'Task: {task}')
            # Get model class name
            model_name = estimator.__class__.__name__
            # Get model attributes
            attributes = estimator.__dict__
            # Convert dictionary to list then to string
            formatted_params = ", ".join([f"{k}='{v}'" if isinstance(v, str) else f"{k}={v}" for k, v in attributes.items()])
            # Concatenate model class name with its attributes
            estimator_name = f"{model_name}({formatted_params})"
            print(f"Model: {estimator_name}")
            print(f'Predictions for {X_new.tolist()}: {np.round(y_pred,2)}')
            print(f'R^2 score on training data: {score_train:.2f} %')
            print('-'*40)    

Task: Classification
Model: Defualt Bagging Classifier
Predictions for [[1, 2], [3, 4], [4, 3]]: [0 1 1]
Accuracy score on training data: 99.00 %
----------------------------------------
Task: Regression
Model: Defualt Bagging Regressor
Predictions for [[1, 2], [3, 4], [4, 3]]: [6.77 9.94 9.94]
R^2 score on training data: 27.52 %
----------------------------------------
Task: Classification
Model: LogisticRegression(method='Gradient Descent', learning_rate=0.1, n_iterations=1000, alpha=1.0, theta=None)
Predictions for [[1, 2], [3, 4], [4, 3]]: [1 1 1]
Accuracy score on training data: 84.00 %
----------------------------------------
Task: Classification
Model: LogisticRegression(method='Ridge', learning_rate=0.1, n_iterations=1000, alpha=1.0, theta=None)
Predictions for [[1, 2], [3, 4], [4, 3]]: [1 1 1]
Accuracy score on training data: 85.00 %
----------------------------------------
Task: Classification
Model: LogisticRegression(method='Lasso', learning_rate=0.1, n_iterations=1000, alp